In [1]:
## READ BOX from csv
import pandas as pd 

In [2]:
# for normal case without adding more negative samples
normal_gt_path = "100GTexist_GroundTruth.csv"
normal_pred_path1 = "100GTexist_Predits.csv"
tn =  45 #for case 2
Sen_name = "Sen"
Spe_name = "Spe"
Acc_name =  "Acc"
Pre_name = "Pre"

# normal_pred_path2 = "100GTexist_Predits2.csv"
# normal_pred_path3 = "100GTexist_Predits3.csv"

# Sen_name = "Sen_plus"
# Spe_name = "Spe_plus"
# Acc_name =  "Acc_plus"
# Pre_name = "Pre_plus"
# tn =  49
# normal_gt_path = "100GT_EXITEGTPLUS_label.csv"
# normal_pred_path1 = "100GT_EXITEGTPLUS_pred.csv"

In [3]:
gt_box = pd.read_csv(normal_gt_path)
pred_box =  pd.read_csv(normal_pred_path1) 

In [4]:
print("gt_box:")
gt_head  = list(gt_box.columns.values)
print(gt_head)
gt_head_int = [int(ele) for ele in list(gt_box.columns.values)]
print(gt_box)
print("gt_head_int:",gt_head_int)
print(gt_box.values[0])
print("pred_box:")
print(pred_box)

gt_box:
['456', '834', '754', '1269']
    456   834  754  1269
0   331  1096  660  1542
1   215    19  503   365
2   220  1217  604  1861
3   272  1351  675  1888
4   418  1253  843  1803
..  ...   ...  ...   ...
94  148   111  325   354
95  152    70  286   212
96  115    97  314   322
97  168   107  380   347
98  138    68  343   376

[99 rows x 4 columns]
gt_head_int: [456, 834, 754, 1269]
[ 331 1096  660 1542]
pred_box:
    450   839  760  1265
0   337  1084  672  1535
1   221    10  518   369
2   229  1205  611  1891
3   281  1358  678  1908
4   414  1263  844  1820
..  ...   ...  ...   ...
94  157   107  328   352
95  153    68  294   219
96  118    93  316   319
97  168   104  386   348
98  141    62  347   375

[99 rows x 4 columns]


In [5]:
print("shape:", gt_box.shape)

shape: (99, 4)


In [6]:
def bb_intersection_over_union(boxA, boxB):
	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(boxA[0], boxB[0])
	yA = max(boxA[1], boxB[1])
	xB = min(boxA[2], boxB[2])
	yB = min(boxA[3], boxB[3])
	# compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
	# compute the area of both the prediction and ground-truth
	# rectangles
	boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
	boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
	# compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction + ground-truth
	# areas - the interesection area
	iou = interArea / float(boxAArea + boxBArea - interArea)
	# return the intersection over union value
	return iou

In [7]:
import numpy as np
IoUs_list = []
FN = 0
for i in range(gt_box.shape[0]+1): # including the first of head
#     BOX1 = gt_box.values[i,:]
#     BOX2 = pred_box.values[i,1:]
    if i == 0:
        BOX1 = [int(ele) for ele in list(gt_box.columns.values)]
        BOX2 = [int(ele) for ele in list(pred_box.columns.values)]
    else:
        BOX1 = gt_box.values[i-1]
        BOX2 = pred_box.values[i-1]
    count_nonzero_pred = np.count_nonzero(BOX2)
    if count_nonzero_pred == 0:
        print("no pred box")
        FN+=1
    else:
    
#     print("BOX1:")
#     print(BOX1)
#     print("BOX2:")
#     print(BOX2)
    # calculate IoU
        temp_IoU =  bb_intersection_over_union(BOX1, BOX2)
        print("temp IoU {}".format(temp_IoU))
        IoUs_list.append(temp_IoU)
print("FN:", FN)

temp IoU 0.9423196150212565
temp IoU 0.9090561273290737
temp IoU 0.8979444413964631
temp IoU 0.9018094196916624
temp IoU 0.9243736365439281
temp IoU 0.9416076287598363
temp IoU 0.9620070719509636
temp IoU 0.9367275842375785
temp IoU 0.908400260473193
temp IoU 0.8426416449672264
temp IoU 0.8705869898904356
temp IoU 0.8826866839272077
temp IoU 0.8560566949828695
temp IoU 0.8820007808771854
temp IoU 0.9457211025302945
temp IoU 0.7902831897261402
temp IoU 0.9091683014398342
temp IoU 0.826911007907619
temp IoU 0.9104280379201379
temp IoU 0.860621493653762
temp IoU 0.9151973825536626
temp IoU 0.8856547760288961
temp IoU 0.8994053300051391
temp IoU 0.9022444345322574
temp IoU 0.9473327347028533
temp IoU 0.7863548317669847
temp IoU 0.8768391898909469
temp IoU 0.9507308718684032
temp IoU 0.9112766082214273
temp IoU 0.9447864879992098
no pred box
temp IoU 0.9134828652650652
temp IoU 0.8358452614837684
temp IoU 0.8819793115963946
temp IoU 0.9518888642979962
temp IoU 0.8874751491053678
temp IoU 0.

In [8]:
def TP_FP1(IoU_list, th):
    TP = 0
 
    FP_GTP =0
    for each_IoU in IoU_list:
        
        if each_IoU>=th:
            TP +=1
        else:
            FP_GTP+=1
    
    return TP, FP_GTP

    

In [9]:
# 
print(IoUs_list)  
print("range of IoU_list: [{}, {}]".format(min(IoUs_list), max(IoUs_list)))
IOU_TH = np.linspace(0.5, 0.95, num=10)
TP_list = []
FP_GTP_LIST =[]
recall_list = []
spec_list = []
acc_list = []
pre_list = []
for each_th  in IOU_TH:
    

    
    TP, FP_GTP  = TP_FP1(IoUs_list, each_th)
    
    
    tp =  TP
    if tn==45:
        fp = FP_GTP + 5
    if tn == 49:
        fp = FP_GTP + 1
    fn = FN
    
#     tn =  49 # for adding more negative plus data
    recall = tp/(tp+fn)
    print("fp:", fp)
    print("tn:", tn)
    specificity = tn/(fp + tn)
    accuracy = (tp+tn)/(tp+fp+tn+fn)
    precision = tp/(tp+fp)
    TP_list.append(TP)
    FP_GTP_LIST.append(FP_GTP)
    recall_list.append(recall)
    spec_list.append(specificity)
    pre_list.append(precision)
    acc_list.append(accuracy)
    print("each th:{} --> TP:{} FP_GTP:{}---> sensitivity:{}, sepcificity: {}, precision:{}, accuracy: {}".format(each_th, TP, FP_GTP, recall,specificity, precision,accuracy))
    
print("AVG TP:", sum(TP_list)/len(TP_list))
print("AVG FP_GTP:", sum(FP_GTP_LIST)/len(FP_GTP_LIST) )
print("AVG sensitivity:", sum(recall_list)/len(recall_list))
print("AVG specificity:", sum(spec_list)/len(spec_list) )
print("AVG precision:", sum(pre_list)/len(pre_list) )
print("AVG accuracy:", sum(acc_list)/len(acc_list) )

[0.9423196150212565, 0.9090561273290737, 0.8979444413964631, 0.9018094196916624, 0.9243736365439281, 0.9416076287598363, 0.9620070719509636, 0.9367275842375785, 0.908400260473193, 0.8426416449672264, 0.8705869898904356, 0.8826866839272077, 0.8560566949828695, 0.8820007808771854, 0.9457211025302945, 0.7902831897261402, 0.9091683014398342, 0.826911007907619, 0.9104280379201379, 0.860621493653762, 0.9151973825536626, 0.8856547760288961, 0.8994053300051391, 0.9022444345322574, 0.9473327347028533, 0.7863548317669847, 0.8768391898909469, 0.9507308718684032, 0.9112766082214273, 0.9447864879992098, 0.9134828652650652, 0.8358452614837684, 0.8819793115963946, 0.9518888642979962, 0.8874751491053678, 0.913370382477934, 0.9193960172149682, 0.9075216270891348, 0.9049962632835712, 0.9671956161105218, 0.8747052450449302, 0.8892994858611826, 0.9116492456132053, 0.8662656177695511, 0.90745794224368, 0.9485820681753079, 0.7879886685552407, 0.8531372977700044, 0.8994906840306879, 0.9365450791465932, 0.627

In [10]:
# SAVE RESULTS TO DISK
import pickle

with open(Sen_name, 'wb') as fp:
    pickle.dump(recall_list, fp)
    
with open(Spe_name, 'wb') as fp:
    pickle.dump(spec_list, fp)

with open(Acc_name, 'wb') as fp:
    pickle.dump(acc_list, fp)
    
with open(Pre_name, 'wb') as fp:
    pickle.dump(pre_list, fp)